In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from collections import defaultdict
import numpy as np
corpus = "I like Natural Language Processing and Neural Networks"

corpus = corpus.split()
word_to_idx = {word: i for i, word in enumerate(set(corpus))}
idx_to_word = {i: word for word, i in word_to_idx.items()}
vocab_size = len(word_to_idx)
def generate_skipgram_data(corpus, window_size):
    data = []
    for i, target_word in enumerate(corpus):
        target_idx = word_to_idx[target_word]
        for j in range(i - window_size, i + window_size + 1):
            if j != i and 0 <= j < len(corpus):
                context_word = corpus[j]
                context_idx = word_to_idx[context_word]
                data.append((target_idx, context_idx))
    return data
window_size = 2
embedding_dim = 50
learning_rate = 0.001
epochs = 1000
skipgram_data = generate_skipgram_data(corpus, window_size)
class SkipGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGram, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)
    def forward(self, target):
        target_embeds = self.embeddings(target)
        output = self.linear(target_embeds)
        return output
model = SkipGram(vocab_size, embedding_dim)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
for epoch in range(epochs):
    total_loss = 0
    for target, context in skipgram_data:
        target_var = Variable(torch.LongTensor([target]))
        context_var = Variable(torch.LongTensor([context]))
        optimizer.zero_grad()
        output = model(target_var)
        loss = criterion(output, context_var)
        loss.backward()
        optimizer.step()
        total_loss += loss.data.item()
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {total_loss:.4f}')
word_embeddings = model.embeddings.weight.data.numpy()
for i in range(vocab_size):
    word = idx_to_word[i]
    embedding = word_embeddings[i]
    print(f'Word: {word}, Embedding: {embedding}')

Epoch [100/1000], Loss: 32.6065
Epoch [200/1000], Loss: 32.1394
Epoch [300/1000], Loss: 32.0085
Epoch [400/1000], Loss: 31.9492
Epoch [500/1000], Loss: 31.9162
Epoch [600/1000], Loss: 31.8955
Epoch [700/1000], Loss: 31.8813
Epoch [800/1000], Loss: 31.8710
Epoch [900/1000], Loss: 31.8633
Epoch [1000/1000], Loss: 31.8573
Word: like, Embedding: [-1.0592855  -0.5826045  -0.44394037  0.8896508   1.6856664   0.39855832
  0.5809075  -0.18489188  0.7459566  -0.06274474  1.9815489   0.58402336
 -0.70412534  1.2052749  -0.3771717  -1.2036979   0.3987278  -0.6125249
 -0.70824     0.5074935   0.33631265 -0.24587905  1.1995177   0.515077
 -2.1858046  -2.0980644   0.31460714  0.98080605  0.0961078   1.2869277
 -2.3101315  -1.2965968   1.9134912   0.88429177  1.6794944  -0.22442059
  2.013761    3.0812306   0.4509621   0.5705621  -0.63860446 -0.6917429
  1.0145663   0.29554993  1.7493105  -0.81911665 -1.3683809  -0.24847715
  2.2202601  -1.9296569 ]
Word: I, Embedding: [ 1.0097629   0.2468085  -0.721